In [ ]:
#import Statements
import pandas as pd
import openpyxl
from openpyxl import load_workbook

In [ ]:
#  Brain  #

#  excel_file-->BOE raw file
#  Output_file--> copy of excel_file--BOE PN-10 digit trimmed
#  excel2--> Cargo (Use Std Name --eg:"Int_Freight.xlsx")
#  excel4--> RM Dimension(Master File)
#  merged_data3--> Complete file(After mapping with cargo and part number drill) ******** Usable file
#  merged_data4--> Only available in cargo ********** Feed to KPI
#  ********************** Appart from mentioned variable rest are intermediate declaration ******************************



In [ ]:
excel_file="BOE-Raw_Data.xlsx"
df=pd.read_excel(excel_file)
df['10Digit PN']=df['Part_Number'].astype(str).str[:10]
output_excel="Copy_of_BOE.xlsx"
df.to_excel(output_excel,index=False)

In [ ]:
path1="Copy_of_BOE.xlsx"
excel1=pd.read_excel(path1)
path2="Int_Freight.xlsx"
excel2=pd.read_excel(path2)

In [ ]:
### first row header from Cargo
excel3=pd.read_excel(path2,header=1)
excel3

In [ ]:
# Filtering without '/' in HAWB
excel3_unique=excel3[~excel3['SHIPMENT_ID'].astype(str).str.contains('/')]
excel3_unique=excel3_unique.drop_duplicates(subset='BOE_NUMBER')
excel3_unique

In [ ]:
# Mapping missed HAWB no with ref to BOE from Cargo
merged_data=pd.merge(excel1,excel3_unique[['BOE_NUMBER','SHIPMENT_ID']],how='left',left_on='BOE',right_on='BOE_NUMBER')
merged_data.drop(columns='BOE_NUMBER',inplace=True)
merged_data['HAWB']=merged_data['HAWB'].fillna(merged_data['SHIPMENT_ID'])
merged_data.drop(columns='SHIPMENT_ID',inplace=True)
merged_data

In [ ]:
# Mapping the required data from cargo
merged_data1=pd.merge(merged_data,excel3_unique[['SHIPMENT_ID','BUSINESS_CASE','ORIGIN_COUNTRY','DESTINATION_COUNTRY','FORWARDER_NAME','FREIGHT_MODE','SERVICE_TYPE_DESCRIPTION','INCOTERM','CHARGEABLE_WEIGHT','VOLUME','COST_CENTER','PAYMENT_AMOUNT_NET']],how='left',left_on='HAWB',right_on='SHIPMENT_ID')
merged_data1.drop(columns='SHIPMENT_ID',inplace=True)

merged_data1

In [ ]:
merged_data1['10Digit PN']=merged_data1['10Digit PN'].astype(object)


In [ ]:
path3="RM Dimension.xlsx"
excel4=pd.read_excel(path3)

In [ ]:
merged_data2=pd.merge(merged_data1,excel4[["PN without '.'",'Pallet/Primary','Working_CBM','Chargeable Wt']],how='left',left_on='10Digit PN',right_on="PN without '.'")
merged_data2.drop(columns="PN without '.'",inplace=True)
merged_data2

In [ ]:
merged_data2['Pallet/Primary']=merged_data2['Pallet/Primary'].astype(float)

In [ ]:
merged_data2['Calc']=merged_data2.apply(lambda row:((row['Working_CBM']/row['Pallet/Primary']) * row['QTY']) if row['FREIGHT_MODE']=='SF'and row['Pallet/Primary']>0 and row['Working_CBM']>0 else ((row['Chargeable Wt']/row['Pallet/Primary'])* row['QTY']) if row['FREIGHT_MODE']=='AF'and row['Pallet/Primary']>0 and row['Chargeable Wt']>0 else row['Chargeable Wt'],axis=1)

In [ ]:
invoice_sum=merged_data2.groupby('HAWB')['Calc'].transform('sum')
invoice_sum

In [ ]:
qty_sum=merged_data2.groupby('HAWB')['QTY'].transform('sum')
qty_sum

In [ ]:
merged_data2['Ratio']=merged_data2['Calc']/invoice_sum
merged_data2

In [ ]:
merged_data2['ratio2']=merged_data2['QTY']/qty_sum
merged_data2

In [ ]:
#  After considering calc and qty
merged_data2['Ratio']=merged_data2['Ratio'].fillna(merged_data2['ratio2'])
# merged_data2.drop(columns='SHIPMENT_ID',inplace=True)
merged_data2

In [ ]:
merged_data2['Per part Price']=merged_data2['Ratio']*merged_data2['PAYMENT_AMOUNT_NET']
merged_data2

In [ ]:
## Copy of Cargo booked

merged_data3=pd.merge(merged_data2,excel3_unique['SHIPMENT_ID'],how='left',left_on='HAWB',right_on='SHIPMENT_ID')
# merged_data2.drop(columns='BOE_NUMBER',inplace=True)
merged_data3
merged_data4=merged_data3[merged_data3['SHIPMENT_ID'].notnull()]
merged_data4

In [ ]:
e_path='Cargo_Map.xlsx'
existing_file=pd.read_excel(e_path)
book=load_workbook(e_path)
writer=pd.ExcelWriter(e_path,engine='openpyxl')
writer.book=book

writer.sheets=dict((ws.title,ws) for ws in book.worksheets)
merged_data4.to_excel(writer,sheet_name='Cargo_Mapped',index=False,header=False,startrow=writer.sheets['Cargo_Mapped'].max_row)
writer.save()